<a href="https://colab.research.google.com/github/dimtr/PyDataEHV_workshop/blob/master/Image%20Generation/PyData_Workshop_Part1_VAE.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>



## Variational Autoencoder (VAE) ( proposed in [this article](https://arxiv.org/abs/1312.6114)) 
<img src="Images/VAE_image.PNG" width="700">

### Variational autoencoder using [Keras](https://keras.io/) with tensorflow in the backend and [tensorflow_probability](https://github.com/tensorflow/probability)


### [Guiding principles](https://keras.io/#guiding-principles) of Keras:
####    *User friendliness: consistent and simple APIs, clear and actionable error messages
####    *Modularity:  a model is a graph of stand alone components (neural layers, cost functions, optimizers, activation functions etc)
####    *Easy extensibility: new modules are simple to add (as classes and functions)
####    *Work with python


### Install packages for in colab

In [ ]:
def run_subprocess_command(cmd):
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    for line in process.stdout:
        print(line.decode().strip())


import sys, subprocess

colab_requirements = [
    #TensorFlow 2.0 (nightly builds)
    "pip install tf-nightly-gpu-2.0-preview==2.0.0.dev20190513",
    #tpf (nightly builds): library for probabilistic reasoning and statistical analysis in TensorFlow
    "pip install tfp-nightly==0.7.0.dev20190508",
]

for i in colab_requirements:
    run_subprocess_command(i)



### Load modules


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import seaborn as sns
import umap
from matplotlib.colors import ListedColormap

#package for progress bar in python
from tqdm.autonotebook import tqdm

from IPython import display

from keras import backend as K
from keras.layers import (Input, InputLayer, Dense, Lambda, Layer, 
                          Add, Multiply)
from keras.models import Model, Sequential
from keras.datasets import fashion_mnist
from keras.utils.vis_utils import model_to_dot, plot_model


import tensorflow_probability as tfp
ds = tfp.distributions

%matplotlib inline


### There are two APIs for defining a model in Keras:
#### * [Sequential model API](https://keras.io/models/sequential/) : easy to define a linear stack of layers quickly. (e.g. the output of one layer is the input of the next layer in stack).
#### * [Functional model API](https://keras.io/getting-started/functional-api-guide/) :more flexible and allows non-linear NN stack

In this workshop, we use the Sequential one because it allows us to quickly define a simple VAE encoder architecture stacking a couple  of convolution layers, flatten them, and connect them with a Dense layer.

In [ ]:
#notebook aesthetics
np.set_printoptions(precision=2,
                    edgeitems=3,
                    linewidth=80,
                    suppress=True)

### Create a fashion-MNIST dataset
The Fashion MNIST dataset was created by e-commerce company, Zalando, as a replacement for MNIST Digits. It is a nice dataset to quickly check and prototype algorithms while it is more complex than simple MNIST.

The Fashion MNIST dataset is identical to the MNIST dataset in terms of training set size, testing set size, number of class labels, and image dimensions:

*   60,000 training example
*   10,000 testing examples
*   10 classes
*   28×28 grayscale images

You cna find a complete description of the dataset [here](https://github.com/zalandoresearch/fashion-mnist)

In [ ]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()


### Visualize some images using matplotlib

In [ ]:
plt.figure(figsize=(7,7))

for i in range(0,16): # how many imgs will show on a grid 
    plt.subplot(4,4, i+1) # open next subplot
    plt.imshow(x_train[i], cmap=plt.get_cmap('gray_r'))
    plt.title(y_train[i]);

In [ ]:
TRAIN_BUF= x_train.shape[0]
BATCH_SIZE=512
TEST_BUF=x_test.shape[0]
DIMS = (x_train.shape[1],x_train.shape[2],1)
N_TRAIN_BATCHES =int(TRAIN_BUF/BATCH_SIZE)
N_TEST_BATCHES = int(TEST_BUF/BATCH_SIZE)

### Data Preprocessing

In [ ]:


# split dataset and normalize
train_images = x_train.reshape(x_train.shape[0],DIMS[0], DIMS[1], 1).astype("float32") / 255.0
test_images = x_test.reshape(x_test.shape[0],DIMS[0], DIMS[1], 1).astype("float32") / 255.0

# batch datasets
# construct a tf Dataset from make a dataset from a numpy array in memory 
# and automatically suffle and batch the dataset with the provided batch size
# tf.Dataset is an efficient way to pass data to tf 
train_dataset = (

    tf.data.Dataset.from_tensor_slices(train_images)
    .shuffle(TRAIN_BUF)
    .batch(BATCH_SIZE)
)
test_dataset = (
    tf.data.Dataset.from_tensor_slices(test_images)
    .shuffle(TEST_BUF)
    .batch(BATCH_SIZE)
)

### Reparameterization trick for VAEs 

To implement encoder and decoder as a neural network, we need to backpropogate through random sampling but backpropogation cannot flow through random node. To overcome this,  VAEs use reparameterization trick: 

$Z$ is approximated with normally distributed $\epsilon$:

$Z\sim N(0,\Sigma) \Rightarrow z = \mu + L\epsilon, \epsilon\sim N(0,1),\Sigma=LL^T$

Now, instead of seeing $Z$ as sampled from $Q(z\m \phi,x)$, we see $Z$ as a function that takes parameter $(\epsilon,( \mu, L))$ and  $\mu$, $L$ come from the encoder. Therefore during backpropogation, all we need is partial derivatives w.r.t. $\mu$, $L$ and $\epsilon$ is irrelevant for taking derivatives.

This trick is shown in the Figure below, taken from [source](https://arxiv.org/pdf/1606.05908.pdf) 

<img align="left" src="Images/Reparameterization_trick.PNG" width="500">



In [ ]:
class VAE(tf.keras.Model):
    """a basic vae class for tensorflow
    Extends: tf.keras.Model
    """

    def __init__(self, **kwargs):
        super(VAE, self).__init__()
        self.__dict__.update(kwargs)
        #encoder 
        self.enc = tf.keras.Sequential(self.enc)
        #decoder
        self.dec = tf.keras.Sequential(self.dec)

    # getting the mulitvariate normal distribution returned by the VAE encoder
    def encode(self, x):
        #Splits a tensor into sub tensors mu and sigma
        mu, sigma = tf.split(self.enc(x), num_or_size_splits=2, axis=1)
        #create and return the multivariate normal distribution on R^k
        return ds.MultivariateNormalDiag(loc=mu, scale_diag=sigma)
    
    # reparameterization trick, sampling from the distribution returned by the encoder
    def reparameterize(self, mean, logvar):
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(logvar * 0.5) + mean

    #take a data point from the latent space and reconstruct its shape
    def reconstruct(self, x):
        mu, _ = tf.split(self.enc(x), num_or_size_splits=2, axis=1)
        return self.decode(mu)
    
    #return the decoded values
    def decode(self, z):
        return self.dec(z)

    #compute the loss function of VAE by estimating:
    # 1. the image loss is computed using the binary cross entropy formula, which can be interpreted as the minus log likelihood of our data
    # 2. the latent loss using the KL divergence formula (for a normal and a standard normal distribution)
    # finally, we get the mean of all the image losses
    # someone can add weight on the two loss functions and optimize it as hyperparameter 
    def compute_loss(self, x):

        q_z = self.encode(x)
        z = q_z.sample()
        x_recon = self.decode(z)
        p_z = ds.MultivariateNormalDiag(
          loc=[0.] * z.shape[-1], scale_diag=[1.] * z.shape[-1]
          )
        kl_div = ds.kl_divergence(q_z, p_z)
        latent_loss = tf.reduce_mean(tf.maximum(kl_div, 0))
        recon_loss = tf.reduce_mean(tf.reduce_sum(tf.math.square(x - x_recon), axis=0))

        return recon_loss, latent_loss

    def compute_gradients(self, x):
        with tf.GradientTape() as tape:
            loss = self.compute_loss(x)
        return tape.gradient(loss, self.trainable_variables)

    @tf.function
    def train(self, train_x):
        gradients = self.compute_gradients(train_x)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

### Define the network architecture
* Quick Recap: Convolutions for encoder. Images from [source1](https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53) and [source2](https://github.com/vdumoulin/conv_arithmetic)

<img align="left" src="Images/1_GcI7G-JLAQiEoCON7xFbhg.gif" width="250"  alt="with filter"  hspace="200"/><img align="left" src="Images/1_1VJDP6qDY9-ExTuQVEOlVg.gif" width="250" hspace="200"/> 


* Deconvolutions (transpose convolutions) for the decoder

<img align="left" src="Images/f2RiP.gif" width="250"  alt="with filter"  hspace="200"/>

In [ ]:
N_Z = 2
encoder = [
    tf.keras.layers.InputLayer(input_shape=DIMS),
    tf.keras.layers.Conv2D(
        filters=32, kernel_size=3, strides=(2, 2), activation="relu"
    ),
    tf.keras.layers.Conv2D(
        filters=64, kernel_size=3, strides=(2, 2), activation="relu"
    ),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=N_Z*2),
]

decoder = [
    tf.keras.layers.Dense(units=7 * 7 * 64, activation="relu"),
    tf.keras.layers.Reshape(target_shape=(7, 7, 64)),
    tf.keras.layers.Conv2DTranspose(
        filters=64, kernel_size=3, strides=(2, 2), padding="SAME", activation="relu"
    ),
    tf.keras.layers.Conv2DTranspose(
        filters=32, kernel_size=3, strides=(2, 2), padding="SAME", activation="relu"
    ),
    tf.keras.layers.Conv2DTranspose(
        filters=1, kernel_size=3, strides=(1, 1), padding="SAME", activation="sigmoid"
    ),
]

###Create the model

In [ ]:
# the optimizer for the model
optimizer = tf.keras.optimizers.Adam(1e-3)
# train the model
model = VAE(
    enc = encoder,
    dec = decoder,
    optimizer = optimizer,
)

In [ ]:
# see datails of the model
model.enc.summary()

### Train the model

In [ ]:
# example data from the test set for plotting results 
example_data = next(iter(test_dataset))


def plot_reconstruction(model, example_data, nex=8, zm=2):

    example_data_reconstructed = model.reconstruct(example_data)
    #and generate images from some random samples
    samples = model.decode(tf.random.normal(shape=(BATCH_SIZE, N_Z)))
    fig, axs = plt.subplots(ncols=nex, nrows=3, figsize=(zm * nex, zm * 3))
    for axi, (dat, lab) in enumerate(
        zip(
            [example_data, example_data_reconstructed, samples],
            ["data", "data recon", "samples"],
        )
    ):
        for ex in range(nex):
            axs[axi, ex].matshow(
                dat.numpy()[ex].squeeze(), cmap=plt.cm.Greys, vmin=0, vmax=1
            )
            axs[axi, ex].axes.get_xaxis().set_ticks([])
            axs[axi, ex].axes.get_yaxis().set_ticks([])
        axs[axi, 0].set_ylabel(lab)

    plt.show()

In [ ]:
# a pandas dataframe to save the loss information to
losses = pd.DataFrame(columns = ['recon_loss', 'latent_loss'])
# a pandas dataframe to save the loss information to
#losses = pd.DataFrame(columns = [ 'recon_loss'])


In [ ]:
n_epochs = 50
for epoch in range(n_epochs):
    # train
    #create the progress bar
    for batch, train_x in tqdm(
        zip(range(N_TRAIN_BATCHES), train_dataset), total=N_TRAIN_BATCHES
    ):
        model.train(train_x)
    # test on holdout
    loss = []
    for batch, test_x in tqdm(
        zip(range(N_TEST_BATCHES), test_dataset), total=N_TEST_BATCHES
    ):
        loss.append(model.compute_loss(train_x))
    losses.loc[len(losses)] = np.mean(loss, axis=0)
    # plot results
    display.clear_output()
    print(
        "Epoch: {} | recon_loss: {} | latent_loss: {}".format(
            epoch, losses.recon_loss.values[-1] , losses.latent_loss.values[-1]
            #"Epoch: {} | recon_loss: {}".format(
            #epoch, losses.recon_loss.values[-1] #, losses.latent_loss.values[-1]
        )
    )
    plot_reconstruction(model, example_data)

### Visualize the latent representation

In [ ]:
q_z = model.encode(train_images)
latent_z = q_z.sample()

In [ ]:
plt.figure(figsize=(15, 8), dpi=80)

colormap = ListedColormap(sns.color_palette(sns.hls_palette(10, l=.45 , s=.8)).as_hex())
plt.scatter(latent_z[:, 0], latent_z[:, 1], c=y_train, cmap=colormap)

cbar = plt.colorbar()
cbar.ax.set_yticklabels(['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal',
                                                     'Shirt', 'Sneaker', 'Bag', 'Ankle boot'])

plt.title('Latent space')

### Visualize the same dataset with UMAP - a k-nn based dimensionality reduction technique.
Umap does not supporting inverse transformation and not data generation but it is optimized for dimensionality reduction

In [ ]:


sns.set(context="paper", style="white")


reducer = umap.UMAP(random_state=42,n_neighbors=5)
embedding = reducer.fit_transform(x_train.reshape(x_train.shape[0],28*28))

plt.figure(figsize=(15, 8), dpi=80)

colormap = ListedColormap(sns.color_palette(sns.hls_palette(10, l=.45 , s=.8)).as_hex())
plt.scatter(embedding[:, 0], embedding[:, 1], c=y_train, cmap=colormap)

cbar = plt.colorbar()
cbar.ax.set_yticklabels(['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal',
                                                     'Shirt', 'Sneaker', 'Bag', 'Ankle boot'])

plt.title('Latent space')

### Generate and morphe images 
show grid in 2D latent space

In [ ]:
# sample from grid
nx = ny =10
meshgrid = np.meshgrid(np.linspace(-3, 3, nx), np.linspace(-3, 3, ny))
meshgrid = np.array(meshgrid).reshape(2, nx*ny).T
x_grid = model.decode(meshgrid)
x_grid = x_grid.numpy().reshape(nx, ny, 28,28, 1)
# fill canvas
canvas = np.zeros((nx*28, ny*28))
for xi in range(nx):
    for yi in range(ny):
        canvas[xi*28:xi*28+28, yi*28:yi*28+28] = x_grid[xi, yi,:,:,:].squeeze()
fig, ax = plt.subplots(figsize=(10,10))
ax.matshow(canvas, cmap=plt.cm.Greys)
ax.axis('off')